Write some markdown to explain that this notebook will be mainly used for the capstone project.
Import the pandas library as pd.
Import the Numpy library as np.
Print the following the statement: Hello Capstone Project Course!

In [ ]:
#url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#from bs4 import BeautifulSoup
#import requests

#r=requests.get(url)
#content=r.content
#soup=BeautifulSoup(content,"html.parser")

In [ ]:
#print(soup)

In [ ]:
#wikitable=soup.find('table',class_='wikitable sortable')
#print(wikitable)
#now the information is stored in wikitable

In [ ]:
#from pandas.io.json import json_normalize
#df = json_normalize(wikitable)

# Step1: read the postal table from an url link (see below)
1. define a URL
2. read the html table into data frame
3. remove the rows of Borough="Not assigned" 
4. reset the index
5. change the value in Neighborhood="Not assigned" to the values of Borough

[The above code does not change anything as in this table, when the "Borough"="Not assigned" is removed, there is no "Neighborhood"="Not assigned"]
[But the code is made and tested to make sure the function is working]
******************************************************************************************************************************************************

In [ ]:
!pip install lxml
import pandas as pd # library for data analsysis

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)[0]
#dfs is a dataframe object

In [ ]:
dfs.head()

In [ ]:
df=dfs[dfs.Borough!="Not assigned"]
df=df.reset_index(drop=True)
df.head()

In [ ]:
for i in range(103):
    value=df.iloc[i]["Neighbourhood"]
    if value=="Not assigned":
        df.iloc[i]["Neighbourhood"]=df.iloc[i]["Borough"]
        print("find a NOT ASSIGNED")
    
df.head(5)
#The above code is not actually used as there is no "Not assigned" value in column "Beighbourhood"

In [ ]:
df.shape

# End of Step1: read the postal table from an url link

# Step 2: read the geo data from an url link and merge it with the table in setp1(see below)
1. define a URL
2. use pandas.read_csv to read the geo data into a data frame
3. merge this data frame with the postal code-borough-neighborhood dataframe

In [ ]:
url="http://cocl.us/Geospatial_data"

import pandas
geo_df = pandas.read_csv(url)

merge_df=(df.join(geo_df.set_index('Postal Code'),on='Postal Code')).copy()

#manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merge_df

In [ ]:
merge_df["Select"]=False
#Add an empty column "Select" to merge_df
merge_df.head()

# End of Step 2: read the geo data from an url link and merge it with the table in setp1

# Step 3: make the GET oepration to retrive the venue info, make clustering and plot on the map (see below)
1. Make query to get the niformation of venues
2. Use the K-means to do the clustering (here I defined only 3 clusters)
3. Plot the vanues on the map with different colors presenting clusters

In [ ]:
rows=merge_df.shape[0]


for i in range(rows):
    string=merge_df.iloc[i]["Borough"]
    if string.find('Toronto')==-1:
        merge_df.iloc[i,5]=False
    else:
        merge_df.iloc[i,5]=True

merge_df.head(10)
# Now all the rows with Borough has word "Toronto" have been marked "True" in column "Select"
# Now merge_df is 103x6

In [ ]:
merge_df.shape

In [ ]:
toronto_df=merge_df[merge_df.Select==True]
toronto_df.reset_index(drop=True)
toronto_df.shape

# Now we have a DataFrame with dimension of 39x6. Menaing 39 neighbourhoods have belong to Borough with "Toronto" word in their name

number_of_neighbourhood=toronto_df.shape[0]
number_of_neighbourhood

In [ ]:
toronto_df.head()

In [ ]:
! pip install folium==0.5.0
import folium

In [ ]:
latitude=43.6548
longitude=-79.3883
map_toronto=folium.Map(location=[latitude,longitude], zoom_start=12)

for lat, lng, borough, postal_code in zip(toronto_df["Latitude"],toronto_df["Longitude"],toronto_df["Borough"],toronto_df["Postal Code"]):
    label='{},{}'.format(postal_code,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto    


In [ ]:
toronto_df.head(3)

In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize

In [ ]:
CLIENT_ID = '4BLKWPUP0EYC2U4UGAI3YQVWR5NDYHVDF1VNHK41DHG1EWVV' # your Foursquare ID
CLIENT_SECRET = '4HYGXEIO145QTOAN4BOHOQNFWDWUSKF3HI3Q2EWZ4GHJIG3J' # your Foursquare Secret
ACCESS_TOKEN = 'XECU25SP52OQUWQJK2DPP42QKAZDRPTHDWOPZCKPJ1DQLGLK' # your FourSquare Access Token
VERSION = '20180605'

neibourghhood_latitude=toronto_df.iloc[0,3]
neibourghhood_longitude=toronto_df.iloc[0,4]
neibourghhood_name=toronto_df.iloc[0,2]
search_query = '' 
radius = 500

latitude=neibourghhood_latitude
longitude=neibourghhood_longitude
LIMIT = 30

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

#Make a query with the information of first neighbourhood

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
#Define a function to get the category type info

In [ ]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues)

#Retrive the venues list from result
#And convert it to a dataframe called "nearby_venues". it is a 30x8 DF

In [ ]:
nearby_venues.head()

In [ ]:
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
filtered_columns
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

#Cut out only the columns needed, add a column "categories" with the clean categories information.
#nearby_venues is now a 30x4 DF
nearby_venues.head()

In [ ]:
nearby_venues.shape

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
#Define a function to search for venues 
#Input: names, latitudes, longitudes, radius
#OutPut: a list of venues with columns "Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category

In [ ]:
toronto_df.head(2)

In [ ]:
names=toronto_df['Neighbourhood']
latitudes=toronto_df['Latitude']
longitudes=toronto_df['Longitude']
toronto_venues=getNearbyVenues(names, latitudes, longitudes, radius=500)
# use the function defined to get the nearby venue information
# toronto_venues is a 855x7 DF wit columns "Neighborhood" , "Neighborhood Latitude", "Neighborhood Longitude", "Venue", "Venue Latitude", "Venue Longitude", "Venue Category"

In [ ]:
toronto_venues.groupby('Neighborhood').count()
# Be careful with the spelling of "Neighbourhood". In the result of query, it is spelled as 'Neighborhood'

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#Now we know there are 196 unique categories of those venues

In [ ]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot is now a 855x196 DF

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#toronto_grouped is a 39 rows × 196 columns DF

In [ ]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#Print out the top 5 venues in the Toronto neighbourhoods

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
#Define a function to calculate the popular of vanues

In [ ]:
import numpy as np

num_top_venues = 6
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

#neighborhoods_venues_sorted is now a 39x7 DF, the column 1 to 6 are the sorted categories of a venue

In [ ]:
from sklearn.cluster import KMeans 

kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:6] 

################################
# manhattan_merged->toronto_merged
# manhattan_data->df
################################
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
toronto_df.shape

In [177]:
toronto_merged = toronto_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

# End of Step 3: make the GET oepration to retrive the venue info, make clustering and plot on the map